In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import pickle

with open('/content/drive/MyDrive/egu23/data/covariates.pkl', 'rb') as f:
    covariates = pickle.load(f)

with open('/content/drive/MyDrive/egu23/data/target.pkl', 'rb') as f:
    target = pickle.load(f)

In [5]:
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46

In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys


import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)


from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler

from darts.models.forecasting.random_forest import RandomForest

# from darts.models import TransformerModel
from darts.metrics import mape, rmse, r2_score


pd.set_option("display.precision",2)
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
len(covariates)

15

In [8]:
i = 0
target_list = []
covar_list = []

for i in range(0, len(covariates)):
  if len(covariates[i]) < 1000:
    i +=1
  
  else:
    target_list.append(TimeSeries.from_dataframe(target[i], time_col = 'TIMESTAMP', freq = '1D'))
    covar_list.append(TimeSeries.from_dataframe(covariates[i], time_col = 'TIMESTAMP', freq = '1D'))
    

In [9]:
len(covar_list)

14

In [12]:
SPLIT = 0.8
NEE_train_list = []
NEE_test_list = [] 

for i in range(0, len(covar_list)):
  NEE_train, NEE_test = target_list[i].split_after(SPLIT)

  NEE_train = NEE_train.astype('float64')
  NEE_test = NEE_test.astype('float64')

  NEE_train_list.append(NEE_train)
  NEE_test_list.append(NEE_test)


In [13]:
scalerNEE = Scaler()
NEE_trtrain_list = []
NEE_trtest_list = []
NEE_tr_list = []

for i in range(0, len(covar_list)):

  scalerNEE.fit_transform(NEE_train_list[i])
  NEE_trtrain = scalerNEE.transform(NEE_train_list[i])
  NEE_trtest = scalerNEE.transform(NEE_test_list[i])
  NEE_tr = scalerNEE.transform(target_list[i])

  
  NEE_tr = NEE_tr.astype(np.float64)
  NEE_trtrain = NEE_trtrain.astype(np.float64)
  NEE_trtest = NEE_trtest.astype(np.float64)

  NEE_trtrain_list.append(NEE_trtrain)
  NEE_trtest_list.append(NEE_trtest)
  NEE_tr_list.append(NEE_tr)

# print("first and last row of scaled price time series:")
# pd.options.display.float_format = '{:,.2f}'.format
# NEE_tr.pd_dataframe().iloc[[0,-1]]

In [14]:
NEE_tr_list[0]

<TimeSeries (DataArray) (TIMESTAMP: 6209, component: 1, sample: 1)>
array([[[0.76]],

       [[0.74]],

       [[0.77]],

       ...,

       [[0.7 ]],

       [[0.67]],

       [[0.74]]])
Coordinates:
  * TIMESTAMP  (TIMESTAMP) datetime64[ns] 2003-01-01 2003-01-02 ... 2019-12-31
  * component  (component) object 'NEE_VUT_REF'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [15]:
covar_train_list = []
covar_test_list = [] 

for i in range(0, len(covar_list)):
  covar_train, covar_test = covar_list[i].split_after(SPLIT)

  covar_train = covar_train.astype('float64')
  covar_test = covar_test.astype('float64')


  covar_train_list.append(covar_train)
  covar_test_list.append(covar_test)

In [16]:
scalerNEE = Scaler()
covar_trtrain_list = []
covar_trtest_list = []
covar_tr_list = []

for i in range(0, len(covar_list)):

  scalerNEE.fit_transform(covar_train_list[i])
  covar_trtrain = scalerNEE.transform(covar_train_list[i])
  covar_trtest = scalerNEE.transform(covar_test_list[i])
  covar_tr = scalerNEE.transform(covar_list[i])

  
  covar_tr = covar_tr.astype(np.float64)
  covar_trtrain = covar_trtrain.astype(np.float64)
  covar_trtest = covar_trtest.astype(np.float64)

  covar_trtrain_list.append(covar_trtrain)
  covar_trtest_list.append(covar_trtest)
  covar_tr_list.append(covar_tr)


In [17]:
covar_tr_list[0]

<TimeSeries (DataArray) (TIMESTAMP: 6209, component: 8, sample: 1)>
array([[[0.45],
        [0.21],
        [0.33],
        ...,
        [0.2 ],
        [0.  ],
        [0.6 ]],

       [[0.43],
        [0.12],
        [0.52],
        ...,
        [0.11],
        [0.  ],
        [0.5 ]],

       [[0.48],
        [0.2 ],
        [0.3 ],
        ...,
...
        ...,
        [0.28],
        [0.  ],
        [0.89]],

       [[0.45],
        [0.2 ],
        [0.29],
        ...,
        [0.28],
        [0.  ],
        [0.92]],

       [[0.54],
        [0.2 ],
        [0.36],
        ...,
        [0.34],
        [0.  ],
        [0.88]]])
Coordinates:
  * TIMESTAMP  (TIMESTAMP) datetime64[ns] 2003-01-01 2003-01-02 ... 2019-12-31
  * component  (component) object 'TA_F' 'SW_IN_F' 'LW_IN_F' ... 'P_F' 'PA_F'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [20]:
model = RandomForest(lags=365, 
                     lags_past_covariates=365, 
                     lags_future_covariates=None, 
                     output_chunk_length=767, 
                     add_encoders={"cyclic": {"future": ["hour", "month"]}})

In [ ]:
model.fit(NEE_trtrain_list, past_covariates=covar_tr_list)

In [ ]:
N_SAMPLES = 100 
N_JOBS = -1 

# testing: generate predictions
pred_list = []

for i in range(0, len(NEE_trtest_list)):
  NEE_pred = model.predict(n=len(NEE_trtest_list[i]),
                           series = NEE_train_list[i],
                           past_covariates=covar_list[i], 
                           num_samples=N_SAMPLES,   
                           n_jobs=N_JOBS, 
                           verbose=True)
  
  pred_list.append(NEE_pred)

import pickle

with open('./rf_predictions.pkl', 'wb') as f:
  pickle.dump(pred_list, f)

with open('./rf_actual.pkl', 'wb') as f:
  pickle.dump(NEE_test_list, f)
     

In [ ]:

for i in range(len(pred_list)):
  RMSE = rmse(NEE_test_list[i], pred_list[i])
  MAPE = mape(NEE_test_list[i], pred_list[i])
  R2 = r2_score(NEE_test_list[i], pred_list[i])
  
  print("RMSE:", f'{RMSE:.2f}')
  print("MAPE:", f'{MAPE:.2f}')
  print("R2:", f'{R2:.2f}')


In [ ]:
NEE_test_list[1].plot()
pred_list[1].plot()